In [1]:
import nltk            # natural language tool kit
import numpy as np     # support for large data structures
import pandas as pd    # data structure support
import re
import time

from importlib import reload 

# POS tagging functions
import pos3

# Load the Penn Treebank Corpus which will serve as our training set.
corpus = nltk.corpus.ptb

In [2]:
train = pd.read_csv('Data/train_split.csv')
train_lite = pd.read_csv('Data/train_lite.csv')
dev = pd.read_csv('Data/dev_split.csv')
del train["Unnamed: 0"]
del dev["Unnamed: 0"]
del train_lite["Unnamed: 0"]
del train_lite["Unnamed: 0.1"]

In [3]:
reload(pos3)
hmm = pos3.HMM()
for sentence in corpus.tagged_sents():
    hmm.update_counts(sentence)
hmm.compute_logprobs()

In [4]:
def tags_as_string(p_sent):
    p = re.compile("[^A-Za-z ]")
    s = p.sub("", p_sent).split()+["?"]
    r = " ".join(hmm.viterbi(s))
    return r

In [5]:
def pretty_timedelta(fmt="%d:%02d:%02d", since=None, until=None):
    """Pretty-print a timedelta, using the given format string."""
    since = since or time.time()
    until = until or time.time()
    delta_s = until - since
    hours, remainder = divmod(delta_s, 3600)
    minutes, seconds = divmod(remainder, 60)
    return fmt % (hours, minutes, seconds)

In [10]:
t0 = time.time()
nrows = train_lite.shape[0]
i = 0
while i < nrows:
    x = train_lite[i:min(i+1000,nrows)]
    f = "Data/train_lite"+str(i)+".df.pkl.gz"
    print("Starting ", f, i, min(i+1000, nrows), pretty_timedelta(since=t0))
    x["question1_pos"] = x["question1"].apply(tags_as_string)
    x["question2_pos"] = x["question2"].apply(tags_as_string)
    x.to_pickle(f)
    print("Finished ", f, i, min(i+1000, nrows), pretty_timedelta(since=t0))
    i += 1000

Starting  Data/train_lite0.df.pkl.gz 0 1000 0:00:00


/Users/delmolino/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/delmolino/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Finished  Data/train_lite0.df.pkl.gz 0 1000 0:01:35
Starting  Data/train_lite1000.df.pkl.gz 1000 2000 0:01:35
Finished  Data/train_lite1000.df.pkl.gz 1000 2000 0:03:08
Starting  Data/train_lite2000.df.pkl.gz 2000 3000 0:03:08
Finished  Data/train_lite2000.df.pkl.gz 2000 3000 0:04:47
Starting  Data/train_lite3000.df.pkl.gz 3000 4000 0:04:47
Finished  Data/train_lite3000.df.pkl.gz 3000 4000 0:06:23
Starting  Data/train_lite4000.df.pkl.gz 4000 5000 0:06:23
Finished  Data/train_lite4000.df.pkl.gz 4000 5000 0:07:56
Starting  Data/train_lite5000.df.pkl.gz 5000 6000 0:07:56
Finished  Data/train_lite5000.df.pkl.gz 5000 6000 0:09:29
Starting  Data/train_lite6000.df.pkl.gz 6000 7000 0:09:29
Finished  Data/train_lite6000.df.pkl.gz 6000 7000 0:11:04
Starting  Data/train_lite7000.df.pkl.gz 7000 8000 0:11:04
Finished  Data/train_lite7000.df.pkl.gz 7000 8000 0:12:39
Starting  Data/train_lite8000.df.pkl.gz 8000 9000 0:12:39
Finished  Data/train_lite8000.df.pkl.gz 8000 9000 0:14:36
Starting  Data/train

In [14]:
y = pd.read_pickle("Data/train_lite8000.df.pkl.gz")
y.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_pos,question2_pos
8000,177687,1919,240809,How could I gain weight in a healthy way?,What are the healthy ways of gaining weight an...,1,WRB MD PRP VB NN IN DT JJ NN .,WP VBP DT JJ NNS IN DT NN CC NNS .
8001,106511,175436,175437,Which is a suitable inpatient drug and alcohol...,Which is a suitable inpatient drug and alcohol...,0,PRP VBZ DT JJ NN NN CC NN IN NN IN NNP NNP . .,PRP VBZ DT JJ NN NN CC NN IN NN IN NNP NNP . .
8002,346033,11511,38523,What open source project did you start contrib...,How can I contribute to open source development?,1,WP JJ NN NN VBD PRP VBP VBG IN .,WRB MD PRP VBP TO VB NN NN .
8003,352045,480927,480928,Why don't boxers fight in cages?,What are shanty towns? How are shanty towns be...,0,WRB PRP MD VB IN NNS .,WP VBP JJ NNS WRB VBP VBN -NONE- VBG VBN .
8004,29658,54848,54849,What is cabinet of ministers?,What does a cabinet minister do?,1,WP VBZ NN IN NNS .,WP VBZ DT NN MD VB .
